In [11]:
import numpy as np
import cv2
from PIL import Image

from Algorithm import permute_rows, repermute_columns, repermute_rows, permute_columns
from Image import blockshaped
from Metrics import Entr, Hist, EQ

In [12]:
def algo(arr):
    # constants
    r = 3.73421
    sum = 0
    for i in range(len(arr)):
        for j in range(len(arr)):
            sum = sum + arr[i,j]
    key = sum / n**2
    lkey = key - int(key)
    
    S_a = np.zeros(n, dtype=float)
    S_b = np.zeros(n, dtype=float)

    S_a[0] = lkey
    S_b[0] = 1 - lkey

    for i in range(1, n):
        S_a[i] =  r*S_a[i-1]*(1 - S_a[i-1])    
        S_b[i] =  r*S_b[i-1]*(1 - S_b[i-1])

    S_1n = np.concatenate((S_a[0:n//2], S_b[n//2:n]))
    S_2n = np.concatenate((S_b[0:n//2], S_a[n//2:n]))
    S_3n = np.concatenate((S_a[0:n//2], S_b[0:n//2]))
    S_4n = np.concatenate((S_b[n//2:n], S_a[n//2:n]))

    img_pi = permute_rows(arr, S_1n, n)
    img_pi = permute_rows(img_pi, S_2n, n)
    img_pi = permute_rows(img_pi, S_3n, n)
    img_pi = permute_rows(img_pi, S_4n, n)

    img_spi = permute_columns(img_pi, S_1n, n)
    img_spi = permute_columns(img_spi, S_2n, n)
    img_spi = permute_columns(img_spi, S_3n, n)
    img_spi = permute_columns(img_spi, S_4n, n)


    # step 4
    img_fpi = permute_rows(img_spi, S_a, n)
    img_fpi = permute_columns(img_fpi, S_b, n)


    # step 5
    L_1 = np.zeros((n**2)//2, dtype=float)
    L_2 = np.zeros((n**2)//2, dtype=float)
    LN_1 = np.zeros((n**2)//2, dtype=int)
    LN_2 = np.zeros((n**2)//2, dtype=int)

    L_1[0] = (1 - lkey)/3
    L_2[0] = (1 - lkey)/4

    for i in range(1, (n**2)//2):
        L_1[i] =  r*L_1[i-1]*(1 - L_1[i-1])    
        L_2[i] =  r*L_2[i-1]*(1 - L_2[i-1])

    for i in range((n**2)//2):
        LN_1[i] = (L_1[i] * (10**14)) % 256   
        LN_2[i] = (L_2[i] * (10**14)) % 256

    # step 6
    img_fpi_str = img_fpi.flatten()

    C = np.zeros(n**2, dtype=np.uint8)
    C[0] = LN_1[0] ^ LN_2[0] ^ img_fpi_str[0]
    C[1] = LN_2[0] ^ C[0] ^ img_fpi_str[1]

    for j in range(1, (n**2)//2):
        C[2*j] = LN_1[j] ^ C[2*j - 1] ^ img_fpi_str[2*j]
        C[2*j + 1] = LN_2[j] ^ C[2*j] ^ img_fpi_str[2*j + 1]
    arr = np.reshape(C, (-1, n))
    return arr

In [9]:
def generateSquareArrayOf256El():
    value_range = [ i for i in range(256)]
    arr1 = np.random.choice(value_range, (16,16))
    return arr1
    
def generatePairArray(diff, arr1):
    arr2 = np.bitwise_xor(diff, arr1)
    return arr2

In [10]:
diff_plain = generateSquareArrayOf256El()
n_pairs = 2
diff_cipher = []

for i in range(n_pairs):
    arr1 = generateSquareArrayOf256El()
    arr2 = generatePairArray(diff_plain, arr1)
    
    arr1_enc = algo(arr1)
    arr2_enc = algo(arr2)
    
    diff_cipher.append(np.bitwise_xor(arr1_enc, arr2_enc))
    
print(diff_cipher)

UnboundLocalError: local variable 'n' referenced before assignment